<a href="https://colab.research.google.com/github/cahcharm/Neuro-and-Complexity-Science/blob/neuro-morphology/Coding/LTP/correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Structural plasticity controlled by calcium based correlation detection**

## Top tips
- Hebbian rule shows the importance of correlation between pre- and post-synapse.
- The correlation is usually represented by spiking-time but structurally effected by three process: synapse formation, potential/depress, synapse pruning.
- the calcium concentration in spines was experimentally shown to be a correlation sensitive signal. NMDA receptors are a candidate mediator for the correlation dependent calcium signal
- Here they establish a quantitative model of correlation detection in synapses based on the calcium influx through NMDA receptors under realistic conditions of irregular pre- and postsynaptic spiking activity with pairwise correlation.
- find a simple threshold mechanism sensitive to correlation, whch mechanism controls synapse formation and elimination. They show that the connectivity structure is shaped by the correlations between neighboring inputs.
Structural plasticity controlled by calcium based correlation detection

## Basic biochemical process

- Newly formed synapses are created in an intermediate, silent state. These frequently encountered silent synapses lack AMPA receptors but have NMDA receptors. The most probable mechanism is the translocation of AMPA receptors into the postsynaptic density (PSD).
- LTP and LTD are in a bistate dependent on correlation made by spiking. The underlying mechanisms may be similar.
- presynapse Ca2+ release, NMDA activate mechanism, Calcium activate CaMKII, CaMKII phosphorylate AMPA receptor to increase AMPA conductivity and insertion.
- CaMKII forms holoenzymes with 6 subunits triggered by different calcium signal level. Short, weak level activate one subunit and deactivate after the level drops; higher level an active subunit can phosphorylate the neiboring subunit and remain active. At resting calcium level, PP1 can dephosphorylate an active subunit but a neighboring active site can immediately rephosphorylate it again. 
- Calcium also activate PP1 which dephosphorylate CaMKII-- long time scale, bistable of LTP and LTD


In [ ]:
# if you want to use or haven't installed brainpy 
# pip install -U brain-py

In [1]:
import numpy as np
import brainpy as bp
import matplotlib.pyplot as plt

In [ ]:
dt=0.5
bp.backend.set(dt=dt)

## Spike Time Dependence of Postsynaptic Calcium Concentration

## Ca2+ Transients Caused by Correlated Irregular Spiking

## A Counter for Correlated Events

## Rate Homeostasis by Synaptic Pruning

## Cooperation and Competition by Spatial Input Correlation